<a href="https://colab.research.google.com/github/TaiseiYamana/optuna_study/blob/main/optuna_tutorial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optuna チュートリアル2

## 参考本
https://www.ohmsha.co.jp/book/9784274230103/
## GitHub
https://github.com/pfnet-research/optuna-book

In [4]:
!pip3 install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.4 MB/s eta 0:00:00


# Studyの保存と再開
Studyはローカルファイルとして保存することが可能です。Studyを保存することで以下のメリットがあります。
## メリット
- 長時間にわたる最適化の途中経過を調べることができる。
- 何らかの原因で最適化が中断された場合に、途中から再開できる。
- 可視化機能などを使い、最適化の傾向を後から分析できる。

In [ ]:
# Optunaをインポート
import optuna

# 目的関数を定義
def objective(trial): # 引数に　trial を指定
    # サジェストAPIで目的関数への入力を定案
    x = trial.suggest_float("x", -4.5, 4.5)
    y = trial.suggest_float("y", -4.5, 4.5)

    # 目的関数の評価値をreturn
    return (1.5 - x + x * y) ** 2 + \
        (2.25 - x + x * y ** 2) ** 2 + \
        (2.625 - x + x * y ** 3) ** 2

In [ ]:
from optuna.study import MaxTrialsCallback

# セーブ先の設定を含めた最適化の実行
storage="sqlite:///optuna.db"
study_name="function_minimize"
study = optuna.create_study(
    direction="minimize",
    study_name=study_name,
    storage=storage, # 保存先を"sqlite:///"から始める文字列を指定
    load_if_exists=True #　既存のstudy_nameのstudyは作成エラーになるため、load_if_exitsをTrueにする。
)

n_trials = 100
study.optimize(objective,
               n_trials=n_trials,
               callbacks=[MaxTrialsCallback(n_trials)])  # MaxTrialsCalbackで再開後の終了trialsを指定

[I 2023-07-02 15:01:33,108] Using an existing study with name 'function_minimize' instead of creating a new one.
[I 2023-07-02 15:01:33,252] Trial 55 finished with value: 14.02432431971972 and parameters: {'x': 3.9687944785349747, 'y': -0.15651512525455474}. Best is trial 29 with value: 0.027249374122802338.
[I 2023-07-02 15:01:33,366] Trial 56 finished with value: 40.26600225783084 and parameters: {'x': 3.57677255307173, 'y': 1.171218633356017}. Best is trial 29 with value: 0.027249374122802338.
[I 2023-07-02 15:01:33,479] Trial 57 finished with value: 200.75663027875908 and parameters: {'x': 2.821380052749278, 'y': 1.6263797008293264}. Best is trial 29 with value: 0.027249374122802338.
[I 2023-07-02 15:01:33,603] Trial 58 finished with value: 0.5062147803757973 and parameters: {'x': 2.681420653375141, 'y': 0.20392405623640375}. Best is trial 29 with value: 0.027249374122802338.
[I 2023-07-02 15:01:33,705] Trial 59 finished with value: 0.48662479923709345 and parameters: {'x': 3.22120

In [ ]:
# Studyのロード
study = optuna.load_study(
    storage=storage,
    study_name=study_name,
)
study.optimize(objective, n_trials=100)

[I 2023-07-02 15:06:29,801] Trial 100 finished with value: 0.09712279700659136 and parameters: {'x': 3.2842807230627313, 'y': 0.6117548319950098}. Best is trial 91 with value: 0.015209582269158257.
[I 2023-07-02 15:06:29,912] Trial 101 finished with value: 0.0921311115979917 and parameters: {'x': 3.5872490543176783, 'y': 0.5779949595899861}. Best is trial 91 with value: 0.015209582269158257.
[I 2023-07-02 15:06:30,025] Trial 102 finished with value: 0.467064037567981 and parameters: {'x': 3.330722223054503, 'y': 0.4452664496009074}. Best is trial 91 with value: 0.015209582269158257.
[I 2023-07-02 15:06:30,133] Trial 103 finished with value: 0.12102922427658372 and parameters: {'x': 3.1970610693637536, 'y': 0.6041638479588184}. Best is trial 91 with value: 0.015209582269158257.
[I 2023-07-02 15:06:30,252] Trial 104 finished with value: 1.7300289317729285 and parameters: {'x': 2.460687556321671, 'y': -0.13597299685584768}. Best is trial 91 with value: 0.015209582269158257.
[I 2023-07-02 

KeyboardInterrupt: ignored

## 探索結果の可視化
最適化の探索結果を可視化することができます。
Optunaの可視化はoptuna.visualizationメソッドを用います。

公式ドキュメント:   
https://optuna.readthedocs.io/en/stable/reference/visualization/index.html  
公式サンプルコード：  
https://github.com/optuna/optuna-examples/blob/main/visualization/plot_study.ipynb


# 探索の効率化

## 1: 最適化アルゴリズムの選択
次に評価すべきパラメータを選択するには、  
サンプラーがパラメータの探索空間から探索点選択することで行われる。

### Samperクラス
Optunaでは探索点選択を行うアルゴリズムはSamperクラスで実装されている。
Optunaのv2.2.0で以下のような**sampler**を利用する事ができる。
詳細は、各々のドキュメントを参照してください。

- [`optuna.samplers.GridSampler`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.samplers.GridSampler.html)
  - グリッドサーチを行うsamplerです。
- [`optuna.samplers.RandomSampler`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.samplers.RandomSampler.html)
  - ランダムサーチを行うsamplerです。
- [`optuna.samplers.TPESampler`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.samplers.TPESampler.html)
  - ベイズ最適化のアルゴリズムの1つであるTPE (Tree-structured Parzen Estimator)を実装しているsamplerです。
  TPEはOptunaのデフォルトのsamplerです。アルゴリズムの詳細は[元論文](https://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf)を参照してください。
- [`optuna.samplers.CmaEsSampler`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.samplers.CmaEsSampler.html)
  - 進化計算のアルゴリズムの1つであるCMA-ES (Covariance Matrix Adaptation - Evolution Strategy) を実装しているsamplerです。
  アルゴリズムの詳細は原著論文の著者によって書かれた[こちらの論文](https://arxiv.org/abs/1604.00772)を参照するとわかりやすいかと思います。
- [`optuna.integration.SkoptSampler`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.integration.SkoptSampler.html)
  - ベイズ最適化の代表的なアルゴリズムである、代理モデル(例えばガウス過程やランダムフォレスト)に基づく最適化を実装しているsamplerです。
  アルゴリズムの実装はサードパーティライブラリの `scikit-optimize` を利用しており、Optunaのbuilt-inのsamplerではありません。
  実装されているアルゴリズムの詳細は[scikit-optimizeの公式ページ](https://scikit-optimize.github.io/stable/auto_examples/bayesian-optimization.html)を参照してください。
  代理モデルに基づく最適化については、[こちらの論文](https://ieeexplore.ieee.org/document/7352306)が網羅的によくまとまっています。


### 指定方法
create_study()の引数samplerで指定  
https://optuna.readthedocs.io/en/stable/reference/generated/optuna.create_study.html  
単目的最適化のデフォルトは***TPESampler***
多目的最適化のデフォルトは***NGSAIISampler***  

### 探索点選択アルゴリズムの使い分け
以下の公式ドキュメントの記載表より最適化対象で対応しているものを選択する。  
https://optuna.readthedocs.io/en/stable/reference/samplers/index.html

最もわかりやすい使い分け基準は、自分の最適化したい目的関数にかけられるトライアル数です。  
例えば、数百トライアル程度ならばデフォルトのTPESamplerを使うが、  
数千トライアルを実行するときはCmaEsSanplerを利用すべき。


|  サンプラー  |  アルゴリズム | 動作速度 | 推奨されるトライアル数 |
| ---- | ---- | ---- | ---- |
|  TPESampler  | ベイズ最適化 | 速い | 最大1000程度 |
|  NSGAIISampler  | 進化計算 | 速い | 最大10000程度 |
| BoTorchSampler | ベイズ最適化 | 遅い | 最大100程度 |
| QMCSampler | 準モンテカルロ法 | 速い | いくらでも |
| cmaEsSampler | 進化計算 | 速い | 最大10000程度 |
| RandomSampler | ランダムサーチ | 速い | いくらでも |
| GridSampler | グリッドサーチ | 速い | 組み合わせの数だけ可能 |


## 2: 枝刈りによるトライアルの早期終了
機械学習では学習時間が長く、1トライアルは設定したエポック分の総学習時間になる。  
そのため枝切りによってハイパーパラメータ調整にかかる実行時間の短縮を行います。  
学習途中の評価位置の変化をみて、望ましい数近いが得られる見込みがない場合は
途中で学習を打ち切ります。  
Optunaでは枝切りを自動的に行うことができます。

### prunerコンポーネント
Optunaでは枝切りの手法がprunerで実装されている。
Optunaのv2.2.0時点で以下のような**pruner**を利用する事ができます。

- [`optuna.pruners.NopPruner`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.NopPruner.html)
  - 何も枝刈りしない、というprunerです。
- [`optuna.pruners.PercentilePruner`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.PercentilePruner.html)
  - 各epochにおいて、最適化履歴の 下位$\alpha$%に入っていれば枝刈りする、というprunerです。
- [`optuna.pruners.MedianPruner`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.MedianPruner.html)
  - 各epochにおいて、最適化履歴の下位50%に入っていれば枝刈りする、というprunerです。
- [`optuna.pruners.SuccessiveHalvingPruner`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.SuccessiveHalvingPruner.html)
  - Successive Halvingという有名な枝刈りアルゴリズムを実装しているprunerです。
  アルゴリズムの詳細は、[こちらの論文](https://arxiv.org/abs/1810.05934)を参照してください。
  なお、Optunaでは実装の都合上、論文とは多少異なるアルゴリズムを実装しています。
- [`optuna.pruners.HyperbandPruner`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.HyperbandPruner.html)
  - Hyperbandという最先端の枝刈りアルゴリズムを実装しているprunerです。
  アルゴリズムの詳細は、[元論文](https://arxiv.org/abs/1603.06560)を参照してください。
  なお、Optunaでは実装の都合上、論文とは多少異なるアルゴリズムを実装しています。
- [`optuna.pruners.ThresholdPruner`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.ThresholdPruner.html)
  - 各epochにおいて、与えられた閾値から外れた値の場合に枝刈りする、というprunerです。

### 指定方法
create_study()の引数prunerで指定  
https://optuna.readthedocs.io/en/stable/reference/generated/optuna.create_study.html  

デフォルトは***BasePruner***

### 枝切りの手順
目的関数の最後に次の手順を追加します。
1.  traial.reportで中間評価値を報告
2.  trial.should_runeで枝刈りを行うべきかの判定

```python
# 1. 中間評価値を報告
trial.report(accuracy, epoch)
# 2. 枝刈りを売るべきか判定する
if trial.should_prune():
  raise optuna.exception.TrialPruned()
```

### 機械学習における枝切り
枝切りを適応した学習率の最適化では学習率が小さいトライアルが枝切りされやすい傾向になる。  
そのため、枝切りを行わない方設定でprunerで***NopPruner***を用います。

一方で、枝切りを多く行い、トライアル数を増やすことが性能改善につながることもよくあります。  
結論、問題の性質に応じて適したアルゴリズムが変化するので、事前知識が必要です。

## 3: 事前知識の適応
実際のユースケースでは事前に良好な探索点が判明している場合があります。    
全てをOptunaに任せるのではなく、ユーザーの事前知識を活用します。

### ユーザーの事前知識
1.  過去の論文や実験で発見され、良好だと反面しているパラメータ
2.  最適化結果の比較対象（ライブラリのデフォルト値、既存研究や運用中のシステムにおける採用値）が使用しているパラメータ
3. 探索空間中で特に重点的に探索したい領域

### study.enqueue_trialメソッド
enqueueメソッドを使うことによって、サンプラーに探索すべき領域のヒントを与えることができる。





In [38]:
import optuna

def objective(trial):
    x = trial.suggest_float("x", -1, 1)
    y = trial.suggest_float("y", -1, 1)
    return x * y

study = optuna.create_study(direction="minimize")

study.optimize(objective, n_trials=5)
for trial in study.trials:
    print(f"[{trial.number}] params={trial.params}, value={trial.value}")

print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")

[I 2023-07-03 16:55:11,884] A new study created in memory with name: no-name-4f135628-c158-4bf5-9cd1-7042dd840ac7
[I 2023-07-03 16:55:11,888] Trial 0 finished with value: -0.518498075523646 and parameters: {'x': -0.530842035689105, 'y': 0.9767464531149368}. Best is trial 0 with value: -0.518498075523646.
[I 2023-07-03 16:55:11,891] Trial 1 finished with value: -0.059192992788933994 and parameters: {'x': -0.09301552436114968, 'y': 0.6363775638043649}. Best is trial 0 with value: -0.518498075523646.
[I 2023-07-03 16:55:11,895] Trial 2 finished with value: 0.2696185125224186 and parameters: {'x': 0.9582142242566596, 'y': 0.28137602813355933}. Best is trial 0 with value: -0.518498075523646.
[I 2023-07-03 16:55:11,899] Trial 3 finished with value: -0.10262101663861367 and parameters: {'x': 0.20915738320178168, 'y': -0.4906401823721971}. Best is trial 0 with value: -0.518498075523646.
[I 2023-07-03 16:55:11,902] Trial 4 finished with value: -0.1315354792856826 and parameters: {'x': -0.416254

[0] params={'x': -0.530842035689105, 'y': 0.9767464531149368}, value=-0.518498075523646
[1] params={'x': -0.09301552436114968, 'y': 0.6363775638043649}, value=-0.059192992788933994
[2] params={'x': 0.9582142242566596, 'y': 0.28137602813355933}, value=0.2696185125224186
[3] params={'x': 0.20915738320178168, 'y': -0.4906401823721971}, value=-0.10262101663861367
[4] params={'x': -0.416254704691317, 'y': 0.3159975798549248}, value=-0.1315354792856826
Best objective value: -0.518498075523646
Best parameter: {'x': -0.530842035689105, 'y': 0.9767464531149368}


In [39]:
import optuna

def objective(trial):
    x = trial.suggest_float("x", -1, 1)
    y = trial.suggest_float("y", -1, 1)
    return x * y

study = optuna.create_study(direction="minimize")

study.enqueue_trial({"x": -0.1})

study.optimize(objective, n_trials=5)
for trial in study.trials:
    print(f"[{trial.number}] params={trial.params}, value={trial.value}")

print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")

[I 2023-07-03 16:55:15,090] A new study created in memory with name: no-name-232c099d-4435-40f5-8a59-e316760e3cbf
[I 2023-07-03 16:55:15,094] Trial 0 finished with value: -0.09363711453586832 and parameters: {'x': -0.1, 'y': 0.9363711453586832}. Best is trial 0 with value: -0.09363711453586832.
[I 2023-07-03 16:55:15,101] Trial 1 finished with value: -0.23744960489770078 and parameters: {'x': 0.7212032250206679, 'y': -0.32924090833190056}. Best is trial 1 with value: -0.23744960489770078.
[I 2023-07-03 16:55:15,109] Trial 2 finished with value: -0.5668024338891039 and parameters: {'x': -0.5965058370522187, 'y': 0.9502043377984337}. Best is trial 2 with value: -0.5668024338891039.
[I 2023-07-03 16:55:15,111] Trial 3 finished with value: -0.26991315760839685 and parameters: {'x': -0.7204962940911606, 'y': 0.3746211601946785}. Best is trial 2 with value: -0.5668024338891039.
[I 2023-07-03 16:55:15,115] Trial 4 finished with value: -0.029673062658218775 and parameters: {'x': 0.446514908649

[0] params={'x': -0.1, 'y': 0.9363711453586832}, value=-0.09363711453586832
[1] params={'x': 0.7212032250206679, 'y': -0.32924090833190056}, value=-0.23744960489770078
[2] params={'x': -0.5965058370522187, 'y': 0.9502043377984337}, value=-0.5668024338891039
[3] params={'x': -0.7204962940911606, 'y': 0.3746211601946785}, value=-0.26991315760839685
[4] params={'x': 0.4465149086493734, 'y': -0.06645480830186479}, value=-0.029673062658218775
Best objective value: -0.5668024338891039
Best parameter: {'x': -0.5965058370522187, 'y': 0.9502043377984337}
